In [ ]:
from math import floor
from sonpy import lib as sp
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns
import neurokit2 as nk
import tempfile
import yaml
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..','..')))
from spike_extraction_trim import *


In [ ]:
# Specify the folder containing your .tsv files
folder_path = ''  # Edit folder path
save_dir = '' # Edit save directory

# Ensure save_dir exists
os.makedirs(save_dir, exist_ok=True)

# Load all TSV files from the specified directory
data = []
file_names = []  # To store the participant IDs based on filenames
task = 'GASTRIC'
df_combined = pd.DataFrame()
df_heart_rate_combined = pd.DataFrame()

for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith("_gastric.smrx"):
            file_path = os.path.join(root, file_name)
            participant_id = file_name.split('_')[0]  # Extract participant ID
            output_file = os.path.join(save_dir, f"{participant_id}_{task}_preprocessed_channel_data.tsv")
            
            # Check if the processed file already exists
            if os.path.exists(output_file):
                print(f"Skipping {file_name}, processed file already exists.")
                continue
            
            print(f"Processing {file_name}...")
            file_names.append(participant_id)
            gastric_preprocessed_file_name, yaml_path = spike_extraction_df(task, file_path, participant_id, save_dir)            

Skipping sub-004_gastric.smrx, processed file already exists.
Skipping sub-006_gastric.smrx, processed file already exists.
Skipping sub-007_gastric.smrx, processed file already exists.
Processing sub-008_gastric.smrx...
Channel data has been saved to /Users/hsavage/OneDrive - University College London/Manuscripts/Gastric_task/Gastric_01_04/physio/extracted//sub-008_channel_data_output.txt
Processing sub-011_gastric.smrx...
Channel data has been saved to /Users/hsavage/OneDrive - University College London/Manuscripts/Gastric_task/Gastric_01_04/physio/extracted//sub-011_channel_data_output.txt
Processing sub-024_gastric.smrx...
Channel data has been saved to /Users/hsavage/OneDrive - University College London/Manuscripts/Gastric_task/Gastric_01_04/physio/extracted//sub-024_channel_data_output.txt
Processing sub-028_gastric.smrx...
Channel data has been saved to /Users/hsavage/OneDrive - University College London/Manuscripts/Gastric_task/Gastric_01_04/physio/extracted//sub-028_channel_da

In [ ]:
def gastric_compute_freq(df, save_dir, figsize=(20,4)):
    try:
        
        #Load channel data
        signal_data = df['stethoscope_amplitude_clean']
        
        # Compute the FFT
        fft_result = np.fft.fft(signal_data)
        
        # Compute the frequencies corresponding to the FFT
        frequencies = np.fft.fftfreq(len(signal_data), d=0.001)  # d= sampling interval
        
        # Compute the amplitude spectrum
        amplitude = np.abs(fft_result)
        
        # Plot frequency vs. amplitude
        plt.figure(figsize=figsize)
        plt.plot(frequencies[:len(frequencies)//2], amplitude[:len(frequencies)//2], lw = 0.2, color='#51C151')  # Plot only positive frequencies
        plt.title("Frequency vs Amplitude")
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Amplitude")
        plt.grid()
        
        plt.tight_layout()

        plt_name = 'GASTRIC_stethoscope_freq.png'
        plt_save_path = save_dir +  '/'+ plt_name
        plt.savefig(plt_save_path, dpi=300, bbox_inches='tight')
        #plt.show()

    
    except Exception as e:
        # Handle exception and create fallback image
        print(f"An error occurred: {e}")
        
        plt_name = 'GASTRIC_stethoscope_freq.png'

        # Create FAILED figure with green shading
        plt.figure(figsize=figsize, facecolor='white')
        plt.gca().set_facecolor('white')
        plt.text(0.5, 0.5, f"Data to produce {plt_name} not found",
                 fontsize=14, ha='center', va='center', color='#ceebc7')

        # Save the fallback image
        fallback_path = 'GASTRIC_stethoscope_freq_FAIL_image.png'
        plt_save_path = save_dir +  '/' + fallback_path
        plt.axis('off') 
        plt.savefig(plt_save_path, bbox_inches='tight', facecolor='white')
    
        return plt_save_path